## **Maze Escape – Save the Scientist( A\* Pathfinding in a Dynamic Environment)**

##  Overview
`MazeEscapeGame` is a Pygame-based simulation where the player helps a scientist escape a 15×15 dynamic maze. It features:
- **Manual and Auto modes** (auto uses A* search).
- Dynamic obstacles (walls, traps, locks).
- Performance tracking (steps, cost, memory, success).
- Visualization with icons and stats panel.

---

## 🧱 Class: `MazeEscapeGame`

### 🎮 Core Attributes
- `screen`, `clock`, `font`, `icons`, `background`: Pygame display/rendering tools.
- `maze`: 15×15 NumPy array of grid cell types:
  - `0`: Empty, `1`: Wall, `2`: Trap, `3`: Lock, `4`: Exit
- `start_pos`, `exit_pos`, `current_pos`, `path`: Position and route data.
- `total_cost`, `steps_taken`, `escape_attempts`, `successful_escapes`
- `control_mode`: `"manual"` or `"auto"`
- `performance_reports`: Tracks steps, cost, time/space complexity.

---

## 🧠 Key Methods

### `_create_icon(color, shape)`
Creates Pygame icon for maze elements (circle, square, triangle, etc.).

### `generate_new_maze()`
Randomly generates a maze with walls (10%), traps (5%), locks (3%), and a reachable exit.

### `heuristic(pos, goal)`
Manhattan distance heuristic for A* pathfinding.

### `calculate_path()`
Uses **A\*** to compute optimal path. Handles:
- Traps (cost 10–20)
- Locks (impassable)
- Returns: max memory used (space complexity)

### `move_scientist(direction=None)`
Moves the scientist:
- **Auto**: Follows precomputed A* path.
- **Manual**: Uses keyboard input.
- Handles traps (cost), locks (fail), and exits (success).
- Introduces dynamic maze updates during auto movement.

### `draw_performance_report(surface, x, y, w, h)`
Displays performance metrics: steps, cost, path depth, memory, success rate.

### `draw_maze()`
Renders grid, icons, UI controls, messages, and performance stats.

### `handle_events()`
Handles key events:
- `M`: Toggle mode
- `Enter`: Compute path
- `Arrow keys`: Manual move
- `Space`: Advance after success/failure
- Returns: `True` to continue game, `False` to exit

### `run()`
Main game loop. Updates game state, processes input, redraws screen at 60 FPS.

---

## 🚀 Usage
- Run the script to launch the game window.
- Use:
  - `Arrow keys` to move (manual)
  - `Enter` to auto-calculate path
  - `M` to toggle control mode
  - `Space` to continue after outcome
- Max 3 attempts per session. Performance summary shown at the end.

---


In [1]:
import numpy as np
import heapq
import time
import random
import pygame
from typing import List, Tuple, Dict, Optional
import pandas as pd  # Added for reporting

# Initialize pygame
pygame.init()

# Constants
WINDOW_WIDTH = 825  # Increased width for better layout
WINDOW_HEIGHT = 800  # Increased height further to make room for controls
GRID_SIZE = 15
FPS = 60
PANEL_HEIGHT = 100  # Increased panel height for controls
SIDEBAR_WIDTH = 200

# Calculate cell size based on available space
CELL_SIZE = min((WINDOW_WIDTH - SIDEBAR_WIDTH - 50) // GRID_SIZE, 
                (WINDOW_HEIGHT - PANEL_HEIGHT - 60) // GRID_SIZE)

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 50, 50)
GREEN = (50, 200, 50)
BLUE = (50, 50, 255)
YELLOW = (255, 255, 0)
PURPLE = (128, 0, 128)
GRAY = (200, 200, 200)
ORANGE = (255, 165, 0)
DARK_GRAY = (50, 50, 50)
LIGHT_BLUE = (173, 216, 230)
BACKGROUND = (240, 248, 255)  

class MazeEscapeGame:
    """
    A game where the player helps a scientist escape from a maze with dual control modes.
    """
    
    def __init__(self):
        """
        Initialize the game with pygame visualization.
        """
        self.screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
        pygame.display.set_caption("Maze Escape - Save the Scientist")
        self.clock = pygame.time.Clock()
        self.font = pygame.font.SysFont('Arial', 18)
        self.title_font = pygame.font.SysFont('Arial', 28, bold=True)
        self.big_font = pygame.font.SysFont('Arial', 36, bold=True)
        
        # Load icons (simple shapes for demonstration)
        self.icons = {
            "scientist": self._create_icon(RED),
            "exit": self._create_icon(GREEN, shape="flag"),
            "wall": self._create_icon(BLACK, shape="square"),
            "trap": self._create_icon(ORANGE, shape="triangle"),
            "lock": self._create_icon(PURPLE, shape="circle"),
            "path": self._create_icon(BLUE, shape="dot")
        }
        
        # Background image (just a gradient for now)
        self.background = pygame.Surface((WINDOW_WIDTH, WINDOW_HEIGHT))
        for y in range(WINDOW_HEIGHT):
            color = (240 - y//10, 248 - y//20, 255)
            pygame.draw.line(self.background, color, (0, y), (WINDOW_WIDTH, y))
        
        self.size = GRID_SIZE
        self.maze = None
        self.start_pos = None
        self.exit_pos = None
        self.path = None
        self.current_pos = None
        self.total_cost = 0
        self.steps_taken = 0
        self.escape_attempts = 0
        self.max_attempts = 3
        self.successful_escapes = 0
        self.game_state = "playing"  # "playing", "success", "failed"
        self.control_mode = "manual" # "auto" or "manual"
        self.message = ""
        self.message_color = BLACK
        
        # Performance tracking
        self.performance_reports = []
        self.current_report = {
            "steps": 0,
            "total_cost": 0,
            "time_complexity": 0,  # Depth of path
            "space_complexity": 0,  # Max memory used
            "success": False
        }
        self.show_performance = False  # Track if we should show performance report
        
        # Calculate maze display area - reduced height to make room for controls
        self.maze_x = 30
        self.maze_y = PANEL_HEIGHT
        self.maze_width = GRID_SIZE * CELL_SIZE
        self.maze_height = GRID_SIZE * CELL_SIZE
        
        self.generate_new_maze()
    
    def _create_icon(self, color, shape="circle"):
        """Create simple icon surfaces for visualization"""
        icon = pygame.Surface((CELL_SIZE-6, CELL_SIZE-6), pygame.SRCALPHA)
        center = (CELL_SIZE//2-3, CELL_SIZE//2-3)
        radius = CELL_SIZE//2-4
        
        if shape == "circle":
            pygame.draw.circle(icon, color, center, radius)
        elif shape == "square":
            pygame.draw.rect(icon, color, (0, 0, CELL_SIZE-6, CELL_SIZE-6))
        elif shape == "triangle":
            points = [(CELL_SIZE//2-3, 0), (0, CELL_SIZE-6), (CELL_SIZE-6, CELL_SIZE-6)]
            pygame.draw.polygon(icon, color, points)
        elif shape == "flag":
            pygame.draw.rect(icon, color, (CELL_SIZE//2-8, 0, 5, CELL_SIZE-6))
            points = [(CELL_SIZE//2-8, 0), (CELL_SIZE-10, CELL_SIZE//3), (CELL_SIZE//2-8, CELL_SIZE//1.5)]
            pygame.draw.polygon(icon, color, points)
        elif shape == "dot":
            pygame.draw.circle(icon, color, center, radius//2)
        return icon
        
    def generate_new_maze(self) -> None:
        """
        Generates a new random maze with different cell types.
        """
        self.maze = np.zeros((self.size, self.size), dtype=int)
        
        # Place walls (10% of cells)
        walls = random.sample(range(self.size * self.size), int(0.1 * self.size * self.size))
        for wall in walls:
            self.maze[wall // self.size][wall % self.size] = 1
            
        # Place traps (5% of cells)
        traps = random.sample(range(self.size * self.size), int(0.05 * self.size * self.size))
        for trap in traps:
            self.maze[trap // self.size][trap % self.size] = 2
            
        # Place locks (3% of cells)
        locks = random.sample(range(self.size * self.size), int(0.03 * self.size * self.size))
        for lock in locks:
            self.maze[lock // self.size][lock % self.size] = 3
            
        # Place exit (1 cell)
        self.exit_pos = (random.randint(0, self.size-1), random.randint(0, self.size-1))
        self.maze[self.exit_pos[0]][self.exit_pos[1]] = 4
        
        # Set start position (must be empty)
        while True:
            self.start_pos = (random.randint(0, self.size-1), random.randint(0, self.size-1))
            if self.maze[self.start_pos[0]][self.start_pos[1]] == 0:
                break
                
        self.current_pos = self.start_pos
        self.path = None
        self.total_cost = 0
        self.steps_taken = 0
        self.game_state = "playing"
        
        # Reset current report
        self.current_report = {
            "steps": 0,
            "total_cost": 0,
            "time_complexity": 0,
            "space_complexity": 0,
            "success": False
        }
        
    @staticmethod
    def heuristic(pos: Tuple[int, int], goal: Tuple[int, int]) -> int:
        """
        Manhattan distance heuristic for A* search.
        """
        return abs(pos[0] - goal[0]) + abs(pos[1] - goal[1])
    
    def calculate_path(self) -> Tuple[int, int]:
        """
        Calculates the optimal path using A* algorithm.
        Returns space complexity (max memory used during search)
        """
        neighbors = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        close_set = set()
        came_from = {}
        gscore = {self.current_pos: 0}
        fscore = {self.current_pos: self.heuristic(self.current_pos, self.exit_pos)}
        open_set = []
        heapq.heappush(open_set, (fscore[self.current_pos], self.current_pos))
        
        max_memory = 0  # Track space complexity
        
        while open_set:
            max_memory = max(max_memory, len(open_set) + len(close_set))
            current = heapq.heappop(open_set)[1]
            
            if current == self.exit_pos:
                path = []
                while current in came_from:
                    path.append(current)
                    current = came_from[current]
                path.append(self.current_pos)
                path.reverse()
                self.path = path
                self.message = "Path found! Press SPACE to move."
                self.message_color = GREEN
                self.current_report["time_complexity"] = len(path)
                self.current_report["space_complexity"] = max_memory
                return max_memory
            
            close_set.add(current)
            
            for dx, dy in neighbors:
                neighbor = (current[0] + dx, current[1] + dy)
                
                if not (0 <= neighbor[0] < self.size and 0 <= neighbor[1] < self.size):
                    continue
                    
                if self.maze[neighbor[0]][neighbor[1]] == 1:  # Wall
                    continue
                if self.maze[neighbor[0]][neighbor[1]] == 3:  # Lock
                    continue
                
                if self.maze[neighbor[0]][neighbor[1]] == 2:  # Trap
                    move_cost = 10 + random.randint(0, 10)
                else:
                    move_cost = 1
                
                tentative_g = gscore[current] + move_cost
                
                if neighbor in close_set and tentative_g >= gscore.get(neighbor, float('inf')):
                    continue
                    
                if tentative_g < gscore.get(neighbor, float('inf')):
                    came_from[neighbor] = current
                    gscore[neighbor] = tentative_g
                    fscore[neighbor] = tentative_g + self.heuristic(neighbor, self.exit_pos)
                    heapq.heappush(open_set, (fscore[neighbor], neighbor))
        
        self.path = None  # No path found
        self.message = "No path found! Try manual mode."
        self.message_color = RED
        return max_memory
    
    def move_scientist(self, direction: Optional[Tuple[int, int]] = None) -> None:
        """
        Moves the scientist either along the calculated path (auto) or in the specified direction (manual).
        """
        if self.control_mode == "auto" and self.path and len(self.path) > 1:
            # Auto movement along calculated path
            next_pos = self.path[1]
        elif self.control_mode == "manual" and direction:
            # Manual movement with arrow keys
            next_pos = (self.current_pos[0] + direction[0], self.current_pos[1] + direction[1])
        else:
            return

        # Check if next position is valid
        if not (0 <= next_pos[0] < self.size and 0 <= next_pos[1] < self.size):
            return
            
        if self.maze[next_pos[0]][next_pos[1]] == 1:  # Wall
            self.message = "Can't move through walls!"
            self.message_color = RED
            return
            
        if self.maze[next_pos[0]][next_pos[1]] == 3:  # Lock
            self.game_state = "failed"
            self.message = "Locked! Escape failed."
            self.message_color = RED
            self.current_report["success"] = False
            self.performance_reports.append(self.current_report.copy())
            self.show_performance = True
            return
            
        # Calculate movement cost
        if self.maze[next_pos[0]][next_pos[1]] == 2:  # Trap
            move_cost = 10 + random.randint(0, 10)
            self.message = f"Trap! Cost: {move_cost}"
            self.message_color = ORANGE
        else:
            move_cost = 1
            self.message = ""
            
        self.total_cost += move_cost
        self.steps_taken += 1
        self.current_pos = next_pos
        
        # Update current report
        self.current_report["steps"] = self.steps_taken
        self.current_report["total_cost"] = self.total_cost
        
        if self.control_mode == "auto":
            self.path.pop(0)
        
        # Check if reached exit
        if self.current_pos == self.exit_pos:
            self.game_state = "success"
            self.successful_escapes += 1
            self.message = "ESCAPE SUCCESSFUL!"
            self.message_color = GREEN
            self.current_report["success"] = True
            self.performance_reports.append(self.current_report.copy())
            self.show_performance = True
            
        # Recalculate path if maze changes dynamically (auto mode only)
        if self.control_mode == "auto" and random.random() < 0.1:  # 10% chance of maze changing
            self.maze[random.randint(0, self.size-1)][random.randint(0, self.size-1)] = random.choice([0, 1, 2, 3])
            self.calculate_path()
    
    def draw_performance_report(self, surface, x, y, width, height):
        """Draws the performance report for the current attempt"""
        report_rect = pygame.Rect(x, y, width, height)
        pygame.draw.rect(surface, (255, 255, 255, 200), report_rect)
        pygame.draw.rect(surface, BLACK, report_rect, 2)
        
        title = self.font.render("PERFORMANCE REPORT", True, BLUE)
        surface.blit(title, (x + width//2 - title.get_width()//2, y + 10))
        
        if not self.performance_reports:
            info = self.font.render("Complete attempts to see reports", True, DARK_GRAY)
            surface.blit(info, (x + width//2 - info.get_width()//2, y + height//2))
            return
        
        y_offset = 40
        for i, report in enumerate(self.performance_reports):
            attempt_text = self.font.render(f"Attempt {i+1}:", True, BLACK)
            surface.blit(attempt_text, (x + 10, y + y_offset))
            
            status = "SUCCESS" if report["success"] else "FAILED"
            status_color = GREEN if report["success"] else RED
            status_text = self.font.render(status, True, status_color)
            surface.blit(status_text, (x + width - status_text.get_width() - 10, y + y_offset))
            
            y_offset += 25
            
            stats = [
                f"Steps: {report['steps']}",
                f"Cost: {report['total_cost']}",
                f"Path Depth: {report['time_complexity']}",
                f"Max Memory: {report['space_complexity']}"
            ]
            
            for stat in stats:
                stat_text = self.font.render(stat, True, DARK_GRAY)
                surface.blit(stat_text, (x + 20, y + y_offset))
                y_offset += 20
            
            y_offset += 10
        
        # Show summary if all attempts are done
        if len(self.performance_reports) >= self.max_attempts:
            y_offset += 10
            summary_title = self.font.render("SUMMARY STATISTICS", True, BLUE)
            surface.blit(summary_title, (x + width//2 - summary_title.get_width()//2, y + y_offset))
            y_offset += 30
            
            successes = sum(1 for r in self.performance_reports if r["success"])
            avg_steps = np.mean([r["steps"] for r in self.performance_reports])
            avg_cost = np.mean([r["total_cost"] for r in self.performance_reports])
            
            summary_stats = [
                f"Success Rate: {successes}/{self.max_attempts}",
                f"Avg Steps: {avg_steps:.1f}",
                f"Avg Cost: {avg_cost:.1f}"
            ]
            
            for stat in summary_stats:
                stat_text = self.font.render(stat, True, DARK_GRAY)
                surface.blit(stat_text, (x + width//2 - stat_text.get_width()//2, y + y_offset))
                y_offset += 25
    
    def draw_maze(self) -> None:
        """
        Draws the maze grid with all elements.
        """
        # Draw background
        self.screen.blit(self.background, (0, 0))
        
        # Draw game title
        title = self.title_font.render("MAZE ESCAPE: SAVE THE SCIENTIST", True, BLACK)
        pygame.draw.rect(self.screen, (255, 255, 255, 180), (0, 0, WINDOW_WIDTH, PANEL_HEIGHT//2))
        self.screen.blit(title, (WINDOW_WIDTH//2 - title.get_width()//2, 15))
        
        # Draw maze container
        maze_rect = pygame.Rect(self.maze_x, self.maze_y, self.maze_width, self.maze_height)
        pygame.draw.rect(self.screen, WHITE, maze_rect)
        pygame.draw.rect(self.screen, BLACK, maze_rect, 3)
        
        # Draw grid lines
        for i in range(self.size + 1):
            pygame.draw.line(self.screen, GRAY, 
                           (self.maze_x, self.maze_y + i * CELL_SIZE), 
                           (self.maze_x + self.maze_width, self.maze_y + i * CELL_SIZE), 1)
            pygame.draw.line(self.screen, GRAY, 
                           (self.maze_x + i * CELL_SIZE, self.maze_y), 
                           (self.maze_x + i * CELL_SIZE, self.maze_y + self.maze_height), 1)
        
        # Draw maze elements
        for y in range(self.size):
            for x in range(self.size):
                cell_x = self.maze_x + x * CELL_SIZE + 3
                cell_y = self.maze_y + y * CELL_SIZE + 3
                rect = pygame.Rect(cell_x, cell_y, CELL_SIZE-6, CELL_SIZE-6)
                
                # Draw cell background
                cell_color = WHITE
                if self.maze[y][x] == 2:  # Trap
                    cell_color = (255, 220, 180)  # Light orange
                elif self.maze[y][x] == 3:  # Lock
                    cell_color = (230, 200, 255)  # Light purple
                pygame.draw.rect(self.screen, cell_color, rect)
                pygame.draw.rect(self.screen, GRAY, rect, 1)
                
                # Draw path (auto mode only)
                if self.control_mode == "auto" and self.path and (y, x) in self.path:
                    self.screen.blit(self.icons["path"], (cell_x, cell_y))
                
                # Draw cell content
                if self.maze[y][x] == 1:  # Wall
                    self.screen.blit(self.icons["wall"], (cell_x, cell_y))
                elif self.maze[y][x] == 2:  # Trap
                    self.screen.blit(self.icons["trap"], (cell_x, cell_y))
                elif self.maze[y][x] == 3:  # Lock
                    self.screen.blit(self.icons["lock"], (cell_x, cell_y))
                elif self.maze[y][x] == 4:  # Exit
                    self.screen.blit(self.icons["exit"], (cell_x, cell_y))
                
                # Draw current position
                if (y, x) == self.current_pos:
                    self.screen.blit(self.icons["scientist"], (cell_x, cell_y))
        
        # Draw sidebar
        sidebar_rect = pygame.Rect(self.maze_x + self.maze_width + 20, self.maze_y, 
                                  SIDEBAR_WIDTH, self.maze_height)
        pygame.draw.rect(self.screen, (255, 255, 255, 200), sidebar_rect)
        pygame.draw.rect(self.screen, BLACK, sidebar_rect, 2)
        
        # Draw legend
        legend_items = [
            ("Scientist", "scientist", RED),
            ("Exit", "exit", GREEN),
            ("Wall", "wall", BLACK),
            ("Trap", "trap", ORANGE),
            ("Lock", "lock", PURPLE),
            ("Path", "path", BLUE)
        ]
        
        legend_y = self.maze_y + 15
        legend_title = self.font.render("LEGEND:", True, BLACK)
        self.screen.blit(legend_title, (sidebar_rect.x + 10, legend_y))
        legend_y += 30
        
        for name, icon_key, color in legend_items:
            self.screen.blit(self.icons[icon_key], (sidebar_rect.x + 20, legend_y))
            text = self.font.render(name, True, color)
            self.screen.blit(text, (sidebar_rect.x + 60, legend_y + CELL_SIZE//2 - 10))
            legend_y += CELL_SIZE + 5
        
        # Draw stats
        stats_y = legend_y + 20
        stats_title = self.font.render("STATS:", True, BLACK)
        self.screen.blit(stats_title, (sidebar_rect.x + 10, stats_y))
        stats_y += 30
        
        stats = [
            f"Attempt: {self.escape_attempts + 1}/{self.max_attempts}",
            f"Steps: {self.steps_taken}",
            f"Total Cost: {self.total_cost}",
            f"Escapes: {self.successful_escapes}"
        ]
        
        for stat in stats:
            text = self.font.render(stat, True, DARK_GRAY)
            self.screen.blit(text, (sidebar_rect.x + 20, stats_y))
            stats_y += 25
        
        # Draw control mode
        mode_y = stats_y + 20
        mode_text = self.font.render(f"Mode: {self.control_mode.upper()}", True, BLUE)
        self.screen.blit(mode_text, (sidebar_rect.x + 20, mode_y))
        mode_hint = self.font.render("(Press M to toggle)", True, DARK_GRAY)
        self.screen.blit(mode_hint, (sidebar_rect.x + 20, mode_y + 25))
        
        # Draw control panel - make it taller to fit all controls
        panel_rect = pygame.Rect(30, self.maze_y + self.maze_height + 20, 
                               WINDOW_WIDTH - 60, PANEL_HEIGHT)
        pygame.draw.rect(self.screen, (255, 255, 255, 200), panel_rect)
        pygame.draw.rect(self.screen, BLACK, panel_rect, 2)
        
        # Draw message
        if self.message:
            message_text = self.font.render(self.message, True, self.message_color)
            self.screen.blit(message_text, (panel_rect.x + 20, panel_rect.y + 15))
        
        # Draw game state message and controls
        if self.game_state == "success":
            text = self.big_font.render("ESCAPE SUCCESSFUL!", True, GREEN)
            self.screen.blit(text, (WINDOW_WIDTH//2 - text.get_width()//2, panel_rect.y + 15))
            
            next_text = self.font.render("Press SPACE to continue", True, BLACK)
            self.screen.blit(next_text, (WINDOW_WIDTH//2 - next_text.get_width()//2, panel_rect.y + 60))
            
        elif self.game_state == "failed":
            text = self.big_font.render("ESCAPE FAILED!", True, RED)
            self.screen.blit(text, (WINDOW_WIDTH//2 - text.get_width()//2, panel_rect.y + 15))
            
            next_text = self.font.render("Press SPACE to continue", True, BLACK)
            self.screen.blit(next_text, (WINDOW_WIDTH//2 - next_text.get_width()//2, panel_rect.y + 60))
            
        else:
            # Draw game controls
            controls_x = panel_rect.x + 20
            controls_y = panel_rect.y + 45
            
            controls_title = self.font.render("CONTROLS:", True, BLACK)
            self.screen.blit(controls_title, (controls_x, controls_y - 30))
            
            # Common controls
            self.screen.blit(self.font.render("M: Switch control mode", True, DARK_GRAY), 
                          (controls_x, controls_y))
            self.screen.blit(self.font.render("SPACE: Continue after success/failure", True, DARK_GRAY), 
                          (controls_x, controls_y + 25))
            
            # Mode-specific controls
            if self.control_mode == "auto":
                self.screen.blit(self.font.render("ENTER: Calculate path", True, BLUE), 
                              (controls_x + 360, controls_y))
                self.screen.blit(self.font.render("SPACE: Move along path", True, BLUE), 
                              (controls_x + 360, controls_y + 25))
            else:
                self.screen.blit(self.font.render("↑ ↓ ← →: Move scientist", True, BLUE), 
                              (controls_x + 360, controls_y))
                self.screen.blit(self.font.render("Arrow keys to navigate maze", True, BLUE), 
                              (controls_x + 360, controls_y + 25))
        
        # Draw performance report if we have any data and it's requested
        if self.performance_reports and self.show_performance:
            report_width = 400
            report_height = 600
            report_x = WINDOW_WIDTH // 2 - report_width // 2
            report_y = WINDOW_HEIGHT // 2 - report_height // 2
            
            # Darken background
            overlay = pygame.Surface((WINDOW_WIDTH, WINDOW_HEIGHT), pygame.SRCALPHA)
            overlay.fill((0, 0, 0, 180))
            self.screen.blit(overlay, (0, 0))
            
            # Draw report
            self.draw_performance_report(self.screen, report_x, report_y, report_width, report_height)
            
            # Draw close button
            close_text = self.font.render("Press SPACE to continue", True, WHITE)
            self.screen.blit(close_text, (WINDOW_WIDTH//2 - close_text.get_width()//2, 
                                       report_y + report_height + 20))
    
    def handle_events(self) -> bool:
        """
        Handles pygame events and returns whether the game should continue.
        """
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return False
                
            if event.type == pygame.KEYDOWN:
                # Handle keys differently when showing performance report
                if self.show_performance:
                    if event.key == pygame.K_SPACE:
                        self.show_performance = False
                        if self.escape_attempts < self.max_attempts - 1:
                            self.escape_attempts += 1
                            self.generate_new_maze()
                        else:
                            return False
                    continue
                
                if event.key == pygame.K_RETURN and self.game_state == "playing" and self.control_mode == "auto":
                    self.calculate_path()
                    
                if event.key == pygame.K_SPACE:
                    if self.game_state == "playing":
                        if self.control_mode == "auto":
                            self.move_scientist()
                    else:
                        # After success/failure, show performance report
                        self.show_performance = True
                
                if event.key == pygame.K_m and self.game_state == "playing":
                    self.control_mode = "manual" if self.control_mode == "auto" else "auto"
                    self.message = f"Switched to {self.control_mode} mode"
                    self.message_color = BLUE
                    if self.control_mode == "auto":
                        self.calculate_path()
                
                # Handle single-step arrow key movement in manual mode
                if self.game_state == "playing" and self.control_mode == "manual":
                    if event.key == pygame.K_UP:
                        self.move_scientist((-1, 0))
                    elif event.key == pygame.K_DOWN:
                        self.move_scientist((1, 0))
                    elif event.key == pygame.K_LEFT:
                        self.move_scientist((0, -1))
                    elif event.key == pygame.K_RIGHT:
                        self.move_scientist((0, 1))
        
        return True
    
    def run(self) -> None:
        """
        Main game loop.
        """
        running = True
        while running:
            running = self.handle_events()
            
            self.draw_maze()
            pygame.display.flip()
            self.clock.tick(FPS)
        
        pygame.quit()

if __name__ == "__main__":
    game = MazeEscapeGame()
    game.run()



pygame 2.6.1 (SDL 2.28.4, Python 3.12.0)
Hello from the pygame community. https://www.pygame.org/contribute.html
